In [46]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [47]:
house_dict = []

for page in range(15, 21):
#     Filter Out Coming Soon, Auction, Home Type: Condos, Days On Zillow: 90 Days

#     Agency Listing
#     linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat1%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'

#     Other Listing
    linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat2%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    print('page: ', page)
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
#             listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
            if (len(loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")) < 3):
                zestimate = 'null'
                rent_zestimate = 'null'
            else:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[0].text
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[2].text
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
#                 "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(5)
            
        except AttributeError as e:
            continue

# print(house_dict)

page:  15
https://www.zillow.com/homedetails/2825-S-Fairview-St-UNIT-F-Santa-Ana-CA-92704/25673931_zpid/
zestimate $406,100
rent zestimate $2,135/mo
https://www.zillow.com/homedetails/10-Partridge-Ln-Aliso-Viejo-CA-92656/25761780_zpid/
zestimate $761,956
rent zestimate $3,234/mo
https://www.zillow.com/homedetails/7758-Caminito-Monarca-UNIT-106-Carlsbad-CA-92009/16661398_zpid/
zestimate $624,000
rent zestimate $2,799/mo
https://www.zillow.com/homedetails/20000-Plum-Canyon-Rd-UNIT-1726-Santa-Clarita-CA-91350/63788552_zpid/
zestimate $466,574
rent zestimate $2,432/mo
https://www.zillow.com/homedetails/15214-S-Raymond-Ave-APT-102-Gardena-CA-90247/20984325_zpid/
zestimate $461,800
rent zestimate $2,589/mo
https://www.zillow.com/homedetails/11124-Burbank-Blvd-APT-303-North-Hollywood-CA-91601/68985542_zpid/
zestimate $524,382
rent zestimate $2,600/mo
https://www.zillow.com/homedetails/20342-Bridgeside-Ln-UNIT-204-Huntington-Beach-CA-92646/25724527_zpid/
zestimate $669,500
rent zestimate $2,69

In [48]:
house_data = pd.DataFrame(house_dict)
house_data

,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,"$395,900","2825 S Fairview St UNIT F, Santa Ana, CA 92704",2825 S Fairview St UNIT F,SantaAna,CA,92704,2,2,796,null,null,Condominium,1974,No Data,$299 monthly,"$406,100","$2,135/mo",$497,https://www.zillow.com/homedetails/2825-S-Fair...
1,"$728,000","10 Partridge Ln, Aliso Viejo, CA 92656",10 Partridge Ln,AlisoViejo,CA,92656,3,3,"1,400",null,null,Condominium,1995,2 Parking spaces,$408 monthly,"$761,956","$3,234/mo",$520,https://www.zillow.com/homedetails/10-Partridg...
2,"$635,000","7758 Caminito Monarca UNIT 106, Carlsbad, CA 9...",7758 Caminito Monarca UNIT 106,Carlsbad,CA,92009,2,3,"1,200",null,null,Condominium,1990,1 Garage space,$390 monthly,"$624,000","$2,799/mo",$529,https://www.zillow.com/homedetails/7758-Camini...
3,"$451,200","20000 Plum Canyon Rd UNIT 1726, Santa Clarita,...",20000 Plum Canyon Rd UNIT 1726,SantaClarita,CA,91350,2,2,"1,001",null,null,Condominium,2001,1 Parking space,$298 monthly,"$466,574","$2,432/mo",$451,https://www.zillow.com/homedetails/20000-Plum-...
4,"$448,900","15214 S Raymond Ave APT 102, Gardena, CA 90247",15214 S Raymond Ave APT 102,Gardena,CA,90247,2,2,"1,045",null,null,Condominium,1978,2 Carport spaces,$290 monthly,"$461,800","$2,589/mo",$430,https://www.zillow.com/homedetails/15214-S-Ray...
5,"$511,300","11124 Burbank Blvd APT 303, North Hollywood, C...",11124 Burbank Blvd APT 303,NorthHollywood,CA,91601,2,2,"1,040",null,null,Condominium,2005,2 Parking spaces,$351 monthly,"$524,382","$2,600/mo",$492,https://www.zillow.com/homedetails/11124-Burba...
6,"$674,900","20342 Bridgeside Ln UNIT 204, Huntington Beach...",20342 Bridgeside Ln UNIT 204,HuntingtonBeach,CA,92646,2,2,"1,104",null,null,Condominium,1987,1 Parking space,$546 monthly,"$669,500","$2,699/mo",$611,https://www.zillow.com/homedetails/20342-Bridg...
7,"$294,900","8901 Salmon Falls Dr APT D, Sacramento, CA 95826",8901 Salmon Falls Dr APT D,Sacramento,CA,95826,2,1,897,null,null,Condominium,1976,1 Carport space,$215 monthly,"$307,200","$1,562/mo",$329,https://www.zillow.com/homedetails/8901-Salmon...
8,"$395,500","26533 Calle San Luis #83B, San Juan Capistrano...",26533 Calle San Luis #83B,SanJuanCapistrano,CA,92675,2,1,897,null,null,Condominium,1971,1 Parking space,$280 monthly,"$398,862","$2,509/mo",$441,https://www.zillow.com/homedetails/26533-Calle...
9,"$395,900","2825 S Fairview St UNIT F, Santa Ana, CA 92704",2825 S Fairview St UNIT F,SantaAna,CA,92704,2,2,796,null,null,Condominium,1974,No Data,$299 monthly,"$406,100","$2,135/mo",$497,https://www.zillow.com/homedetails/2825-S-Fair...


In [49]:
# Export file as a CSV, without the Pandas index, but with the header
# house_data.to_csv("Output/Zillow/Q2-2021_Condo_Data_AgentListing_page15-20.csv", index=False, header=True)
house_data.to_csv("Output/Zillow/Q2-2021_Condo_Data_OtherListing_page15-20.csv", index=False, header=True)